# <div style='color:#fe618e;font-weight:800;'>optuna库选择超参数</div>


In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback

import optuna
from stable_baselines3.common.evaluation import evaluate_policy



In [2]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')


In [3]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [4]:
# from stable_baselines3.common.evaluation import evaluate_policy


In [5]:
def optimize_function(trial):
    
    try:
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)

        monitor_dir = r'./monitor_log/'
        os.makedirs(monitor_dir,exist_ok=True)
        env = Monitor(env,monitor_dir)

        env = GrayScaleObservation(env,keep_dim=True)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env,4,channels_order='last')


        model_params = optimize_ppo(trial) 
        # model_params = {
        #     'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        #     'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        #     'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        #     'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        #     'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
        # }

        tensorboard_log = r'./tensorboard_log/'
        model = PPO("CnnPolicy", env, verbose=0,tensorboard_log=tensorboard_log,**model_params)
        model.learn(total_timesteps=1000)
        # model.learn(total_timesteps=200000)
    
        # mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
        mean_reward, _ = evaluate_policy(model, env,n_eval_episodes=1)
    
    
        env.close()
    
        save_model_dir = r'F:\\RL_Mario1\\'
        os.makedirs(save_model_dir,exist_ok=True)
        SAVE_PATH = os.path.join(save_model_dir, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)
    
        return mean_reward    

    except Exception as e:
        return -1000
    
    

In [6]:
# import optuna

In [7]:
study = optuna.create_study(direction='maximize')
# study.optimize(optimize_function, n_trials=100)
study.optimize(optimize_function, n_trials=3)

[I 2022-03-10 10:24:43,542] A new study created in memory with name: no-name-bc01070f-8a31-4b43-90a9-69370b42bb39
D:\software\e_anaconda\envs\pytorch\lib\site-packages\stable_baselines3\ppo\ppo.py:137: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3736`, after every 58 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3736 and n_envs=1)
  warnings.warn(
D:\software\e_anaconda\envs\pytorch\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
[I 2022-03-10 10:28:37,340] Trial 0 finished with value: -1260.0 and parameters: {'n_steps': 3736, 'gamma': 0.8113866241862995, 'learning_rate': 7.613128199159764e-05, 'clip_range': 0.29279385654405704, 'gae_lambda': 0.937024171211452}. Best is trial 0 with val

In [8]:
study.best_params

{'n_steps': 2592,
 'gamma': 0.8450658799536929,
 'learning_rate': 2.4952570083224206e-05,
 'clip_range': 0.3315680151889212,
 'gae_lambda': 0.9606306727084878}

In [9]:
study.best_trial

FrozenTrial(number=1, values=[680.0], datetime_start=datetime.datetime(2022, 3, 10, 10, 28, 37, 341786), datetime_complete=datetime.datetime(2022, 3, 10, 10, 29, 31, 487396), params={'n_steps': 2592, 'gamma': 0.8450658799536929, 'learning_rate': 2.4952570083224206e-05, 'clip_range': 0.3315680151889212, 'gae_lambda': 0.9606306727084878}, distributions={'n_steps': IntUniformDistribution(high=8192, low=2048, step=1), 'gamma': LogUniformDistribution(high=0.9999, low=0.8), 'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05), 'clip_range': UniformDistribution(high=0.4, low=0.1), 'gae_lambda': UniformDistribution(high=0.99, low=0.8)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)